In [1]:
# Data Preparation, Training and evaluation
import numpy as np
import cv2

image = cv2.imread('images/digits.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
small = cv2.pyrDown(image)
cv2.imshow('Digits Image', small)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Split the image to 500 cells, each 20 x 20 size
cells = [np.hsplit(row,100) for row in np.vsplit(gray, 50)]


# convert the list data type into a numpy array of shape (50, 100, 20, 20)
x = np.array(cells)
print("The shape of our cells array: " + str(x.shape))